# Pandas Advanced

This notebook builds on pandas fundamentals and focuses on advanced indexing, grouping, time series, and performance patterns.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 30)


## 1. Index Mastery (Beyond Basics)
MultiIndex creation/manipulation, index alignment, and cross-section slicing.


In [ ]:
# MultiIndex creation
idx = pd.MultiIndex.from_product(
    [["US", "CA"], ["A", "B", "C"]],
    names=["country", "segment"]
)

s = pd.Series(np.arange(len(idx)), index=idx)
s


In [ ]:
# set_index / reset_index

df = pd.DataFrame({
    "country": ["US", "US", "CA", "CA"],
    "segment": ["A", "B", "A", "B"],
    "value": [10, 20, 15, 25]
})

mi = df.set_index(["country", "segment"])
mi


In [ ]:
mi.reset_index()


In [ ]:
# swaplevel / reorder_levels
mi_swapped = mi.swaplevel(0, 1)
mi_swapped


In [ ]:
mi_reordered = mi.reorder_levels([1, 0])
mi_reordered


In [ ]:
# .xs() cross-section slicing
mi.xs("US", level="country")


In [ ]:
# Index alignment rules: operations align on labels
left = pd.Series([1, 2, 3], index=["a", "b", "c"])
right = pd.Series([10, 20, 30], index=["b", "c", "d"])

left + right  # aligns on index labels, produces NaN where missing


## 2. Advanced Groupby Patterns
Named aggregations, transform vs apply vs agg, and group-wise normalization.


In [ ]:
df = pd.DataFrame({
    "team": ["A", "A", "B", "B", "B"],
    "player": ["p1", "p2", "p3", "p4", "p5"],
    "points": [10, 15, 7, 12, 20],
    "assists": [3, 5, 2, 4, 6]
})

df


In [ ]:
# Named aggregations
agg = df.groupby("team").agg(
    points_mean=("points", "mean"),
    points_max=("points", "max"),
    assists_sum=("assists", "sum")
)
agg


In [ ]:
# Multiple aggregations per column
multi_agg = df.groupby("team")["points"].agg(["mean", "min", "max"]) 
multi_agg


In [ ]:
# transform (same shape as original) vs apply vs agg

# Group-wise z-score (transform)
points_z = df.groupby("team")["points"].transform(lambda x: (x - x.mean()) / x.std())

# apply is flexible but slower (returns arbitrary shape)
apply_example = df.groupby("team").apply(lambda g: g.nlargest(1, "points"))

points_z, apply_example


In [ ]:
# Group-wise normalization and ranking

df["points_rank"] = df.groupby("team")["points"].rank(ascending=False)
df["points_share"] = df["points"] / df.groupby("team")["points"].transform("sum")

df


In [ ]:
# Time-based groupby (year, month)

dates = pd.date_range("2024-01-01", periods=8, freq="D")
values = np.arange(8) * 10

ts = pd.DataFrame({"date": dates, "value": values}).set_index("date")

ts.groupby(ts.index.month).mean()


## 3. Window / Rolling / Expanding Ops
rolling, expanding, ewm, grouped rolling, centered windows.


In [ ]:
s = pd.Series([1, 2, 3, 4, 5, 6])

s.rolling(window=3).mean()


In [ ]:
# Centered vs trailing windows
s.rolling(window=3, center=True).mean()


In [ ]:
# Expanding (cumulative)
s.expanding().mean()


In [ ]:
# Exponentially weighted moving average
s.ewm(alpha=0.3).mean()


In [ ]:
# Custom rolling function
s.rolling(window=3).apply(lambda x: x.max() - x.min())


In [ ]:
# Grouped rolling

df = pd.DataFrame({
    "team": ["A", "A", "A", "B", "B", "B"],
    "game": [1, 2, 3, 1, 2, 3],
    "points": [10, 12, 9, 7, 14, 11]
})

df["rolling_points"] = (
    df.groupby("team")["points"]
      .rolling(window=2, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

df


## 4. Time Series Internals
DatetimeIndex, PeriodIndex, TimedeltaIndex, resampling, time zones.


In [ ]:
# DatetimeIndex
rng = pd.date_range("2024-01-01", periods=5, freq="D")

ts = pd.Series([5, 3, 6, 2, 7], index=rng)

ts


In [ ]:
# Resampling vs grouping

ts.resample("2D").mean()


In [ ]:
# PeriodIndex
pi = pd.period_range("2024Q1", periods=4, freq="Q")
pi


In [ ]:
# TimedeltaIndex

durations = pd.to_timedelta(["1D", "2D", "3D"])
durations


In [ ]:
# Time-zone aware data

tz_rng = pd.date_range("2024-01-01", periods=3, freq="H", tz="UTC")
tz_series = pd.Series([1, 2, 3], index=tz_rng)

# Convert to US/Eastern

tz_series.tz_convert("US/Eastern")


In [ ]:
# Offsets and business calendars
biz = pd.date_range("2024-01-01", periods=5, freq="B")
biz


In [ ]:
# Irregular time series handling

irregular = pd.Series([1, 4, 2], index=pd.to_datetime(["2024-01-01", "2024-01-03", "2024-01-10"]))

# Reindex to daily, fill missing
irregular.reindex(pd.date_range("2024-01-01", "2024-01-10"), method="ffill")


## 5. Advanced Joins & Reshaping
merge_asof, joins on index vs columns, pivot vs pivot_table, melt/stack.


In [ ]:
# merge_asof: nearest key match (sorted keys required)

left = pd.DataFrame({
    "time": pd.to_datetime(["2024-01-01 09:00", "2024-01-01 09:05", "2024-01-01 09:10"]),
    "price": [100, 101, 102]
}).sort_values("time")

right = pd.DataFrame({
    "time": pd.to_datetime(["2024-01-01 09:02", "2024-01-01 09:07"]),
    "event": ["A", "B"]
}).sort_values("time")

pd.merge_asof(left, right, on="time")


In [ ]:
# Join on index vs columns

left = pd.DataFrame({"id": [1, 2, 3], "name": ["Ana", "Ben", "Cara"]}).set_index("id")
right = pd.DataFrame({"id": [2, 3, 4], "team": ["X", "Y", "Z"]}).set_index("id")

left.join(right, how="left")


In [ ]:
# Many-to-many join explosion risk

left = pd.DataFrame({"key": [1, 1], "val_l": ["a", "b"]})
right = pd.DataFrame({"key": [1, 1, 1], "val_r": ["x", "y", "z"]})

pd.merge(left, right, on="key")  # 2 x 3 -> 6 rows


In [ ]:
# pivot vs pivot_table

df = pd.DataFrame({
    "city": ["A", "A", "B", "B"],
    "year": [2023, 2024, 2023, 2024],
    "value": [10, 12, 8, 9]
})

pivoted = df.pivot(index="city", columns="year", values="value")
pivoted


In [ ]:
# pivot_table supports aggregation

df_dup = pd.DataFrame({
    "city": ["A", "A", "A"],
    "year": [2024, 2024, 2024],
    "value": [10, 12, 14]
})

pd.pivot_table(df_dup, index="city", columns="year", values="value", aggfunc="mean")


In [ ]:
# melt / stack / unstack

df = pd.DataFrame({
    "id": [1, 2],
    "x": [10, 20],
    "y": [30, 40]
})

melted = df.melt(id_vars=["id"], value_vars=["x", "y"], var_name="var", value_name="val")

melted


In [ ]:
stacked = df.set_index("id").stack()
stacked


In [ ]:
unstacked = stacked.unstack()
unstacked


## 6. Memory & Performance
Categorical dtype, nullable dtypes, chunked processing, vectorization patterns.


In [ ]:
# Copy vs view semantics (pandas often copies)

df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})

slice_df = df[["a", "b"]]    # likely a new object
slice_df is df


In [ ]:
# Categorical dtype to save memory

cities = pd.Series(["Austin", "Austin", "Boston", "Austin", "Boston"], dtype="category")
cities.dtype


In [ ]:
# Nullable dtypes

s = pd.Series([1, None, 3], dtype="Int64")
s


In [ ]:
# Chunked processing

csv_path = "big_sample.csv"

pd.DataFrame({"x": np.arange(1000), "y": np.random.randn(1000)}).to_csv(csv_path, index=False)

chunk_sums = []
for chunk in pd.read_csv(csv_path, chunksize=200):
    chunk_sums.append(chunk["y"].sum())

sum(chunk_sums)


In [ ]:
# Vectorization pattern

df = pd.DataFrame({"x": np.arange(5), "y": np.arange(5, 10)})

# Avoid loops, use vectorized ops

df["z"] = df["x"] * 2 + df["y"]

df


## 7. Advanced Indexing & Assignment
Boolean indexing pitfalls, chained assignment, .loc vs .iloc, masked assignment.


In [ ]:
df = pd.DataFrame({"a": [1, 2, 3, 4], "b": [10, 20, 30, 40]})

# Boolean indexing
mask = df["a"] % 2 == 0

filtered = df[mask]
filtered


In [ ]:
# Chained assignment (avoid)
# df[mask]["b"] = 999  # SettingWithCopyWarning

# Use .loc for safe mutation

df.loc[mask, "b"] = 999

df


In [ ]:
# Index alignment during assignment

s = pd.Series([100, 200], index=[0, 3])

df["b"] = s  # aligns on index, introduces NaN for missing

df


## 8. Custom Functions at Scale
Use NumPy inside pandas pipelines and avoid Python loops.


In [ ]:
df = pd.DataFrame({"a": np.arange(5), "b": np.arange(5, 10)})

def fast_fn(x):
    # vectorized inside pandas
    return np.sqrt(x**2 + 1)

out = df["a"].pipe(fast_fn)
out


In [ ]:
# Broadcasting tricks with DataFrames

A = pd.DataFrame(np.arange(6).reshape(3, 2), columns=["x", "y"])

# Subtract column means (broadcast across rows)
A - A.mean(axis=0)


## 9. Data Validation & Consistency
Schema checks, duplicate detection, integrity checks after joins.


In [ ]:
df = pd.DataFrame({
    "id": [1, 2, 2, 3],
    "value": [10, 20, 20, 30]
})

# Detect duplicates on multiple keys

df.duplicated(subset=["id", "value"], keep=False)


In [ ]:
# Integrity check after join

left = pd.DataFrame({"id": [1, 2, 3], "x": [10, 20, 30]})
right = pd.DataFrame({"id": [2, 3], "y": [200, 300]})

joined = left.merge(right, on="id", how="left", validate="one_to_one")
joined


In [ ]:
# Missing-data propagation

s = pd.Series([1.0, np.nan, 3.0])

s + 1  # NaN propagates


## 10. Interoperability & Pipelines
Pandas <-> NumPy <-> matplotlib, scikit-learn, and method chaining.


In [ ]:
# Pandas -> NumPy

df = pd.DataFrame({"x": [1, 2, 3], "y": [4, 5, 6]})
arr = df.to_numpy()
arr


In [ ]:
# Pandas -> matplotlib

ax = df.plot(kind="line", x="x", y="y", title="Simple Line")
plt.show()


In [ ]:
# Pandas -> scikit-learn (if installed)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled = scaler.fit_transform(df[["x", "y"]])
# scaled


In [ ]:
# Method chaining with .pipe

def add_ratio(df):
    return df.assign(ratio=df["y"] / df["x"])

chained = (
    df
    .query("x > 1")
    .pipe(add_ratio)
    .sort_values("ratio", ascending=False)
)

chained
